# Questionnair preparation

In [9]:
from topics_postprocessing import Unlemmatizer, TopicsLoader
from utils import load

In [3]:
# GET DEWAC1 TOP 20 TOPIC TERMS

dataset = 'dewac1'
param_ids = 'e42'
nbs_topics = 100
version = 'noun'
topn = 20

topics_loader = TopicsLoader(
    dataset=dataset,
    param_ids=param_ids,
    nbs_topics=nbs_topics,
    version=version,
    topn=topn
)

loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/dewac1_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/dewac1_LDAmodel_e42_100_ep30


In [4]:
tpx = topics_loader.topics
tpx

term0           term1           ...                        term18                  term19
dataset param_id nb_topics topic_idx                                                ...                                                      
dewac1  e42      100       0                       Partei             PDS           ...                      Homepage                  Stalin
                           1                        König            Frau           ...                    Prinzessin                   Nacht
                           2          Lebenspartnerschaft   Lebenspartner           ...                     Erklärung          Homosexualität
                           3                       BetrVG     Betriebsrat           ...                           BAG                     aaO
                           4                 Gesellschaft    Aufsichtsrat           ...                       Satzung               Wahllokal
                           5                      Artikel          Gesetz           ...                   Abgeordnete                 Vertrag
                           6                       Kirche         Bischof           ...            Johannes_Paul_II_.                  Dienst
                           7                 Entscheidung       Verfahren           ...                         Frist           Voraussetzung
                           8                      Richter             Dr.           ...                      Tatsache              Verbrechen
                           9                  Zivildienst     Wehrpflicht           ...                   Grundgesetz                 Einsatz
                           10                      Mensch         München           ...                         Frage           Veranstaltung
                           11                      Physik             Bio           ...                          Land             Deutschland
                           12                       Musik            Band           ...                       Melodie                     Hit
                           13                     Theater           Stück           ...                        August            Käfighaltung
                           14                 Unternehmen           Kunde           ...                       Zukunft                Anbieter
                           15                       Regie           Thoma           ...                         Platz               Armstrong
                           16                         SPD           Grüne           ...                     Politiker                 CDU/CSU
                           17                Rechtsanwalt            Erbe           ...            Patientenverfügung           Maria_Altmann
                           18                       Leute            Zeit           ...                        Moment                    Frau
                           19                  Richtlinie        Regelung           ...                   Gesetzgeber                Maßnahme
                           20                   Bewerbung           Frage           ...                   Anschreiben                    Lena
                           21                       Spiel         Spieler           ...                        Würfel                Beispiel
                           22                        Frau           Vater           ...                          Ende                   Paris
                           23                         Dr.           Prof.           ...                          Zeit             Ausstellung
                           24                        Tier            Hund           ...                         Teich                  Futter
                           25                      Mensch           Leben           ...                        Friede                   Islam
                           26                        Land           Stadt   

In [11]:
import json
from os.path import join

import pandas as pd
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import LdaModel

from constants import LDA_PATH, BAD_TOKENS, PLACEHOLDER, DSETS, NOUN_PATTERN
from utils import load, tprint


class Unlemmatizer(object):

    def __init__(self):
        self.phrases = load('phrases', 'lemmap')
        self.wiktionary = load('wikt', 'lemmap')

    def unlemmatize_token(self, token, lemmap=None):
        print(token)
        # 1) unlemmatize from original dataset
        if lemmap is not None and token in lemmap:
            print(1, 'token in lemmap')
            word = lemmap[token]
        # 2) unlemmatize from Wikipedia title phrases
        elif token in self.phrases:
            print(2, 'token in phrases')
            word = self.phrases[token]
        # 3) unlemmatize individual parts of a concatenated token
        elif '_' in token:
            print(3, 'is phrase')
            tokens = token.split('_')
            ts = []
            for t in tokens:
                print(t)
                if t in self.wiktionary:
                    print('token in wikt')
                    print(self.wiktionary.loc[t])
                    ts.append(t)
                elif t.title() in self.wiktionary:
                    print('token.lower in wikt')
                    print(self.wiktionary.loc[t.title()])
                    ts.append(t)
                else:
                    ts.append(t)
            word = '_'.join(ts)
            print(word)
        else:
            print(4, 'not in index')
            word = token
        # remove concatenation characters
        word = word.replace('_.', '.').replace('_', ' ')
        print('   ', token, '->', word)
        return word

    def unlemmatize_group(self, group):
        lemmap = load(group.name, 'lemmap')
        return group.applymap(lambda x: self.unlemmatize_token(x, lemmap))

    def unlemmatize_topics(self, topics, dataset=None):
        topics = topics.copy()
        if dataset is not None:
            lemmap = load(dataset, 'lemmap')
            topics = topics.applymap(lambda x: self.unlemmatize_token(x, lemmap))
        else:
            topics = topics.groupby('dataset', sort=False).apply(self.unlemmatize_group)
        return topics

ul = Unlemmatizer()

Reading ../data/preprocessed/dewiki_phrases_lemmatized.pickle
Reading ../data/preprocessed/wiktionary_lemmatization_map.pickle
Reading ../data/preprocessed/dewac1_lemmatization_map.pickle


In [15]:
df_unlemm = ul.unlemmatize_topics(topics=tpx, dataset=dataset)
df_unlemm

Reading ../data/preprocessed/dewac1_lemmatization_map.pickle
Partei
1 token in lemmap
    Partei -> Partei
König
1 token in lemmap
    König -> König
Lebenspartnerschaft
1 token in lemmap
    Lebenspartnerschaft -> Lebenspartnerschaft
BetrVG
1 token in lemmap
    BetrVG -> BetrVG
Gesellschaft
1 token in lemmap
    Gesellschaft -> Gesellschaft
Artikel
1 token in lemmap
    Artikel -> Artikel
Kirche
1 token in lemmap
    Kirche -> Kirche
Entscheidung
1 token in lemmap
    Entscheidung -> Entscheidung
Richter
1 token in lemmap
    Richter -> Richter
Zivildienst
1 token in lemmap
    Zivildienst -> Zivildienst
Mensch
1 token in lemmap
    Mensch -> Menschen
Physik
1 token in lemmap
    Physik -> Physik
Musik
1 token in lemmap
    Musik -> Musik
Theater
1 token in lemmap
    Theater -> Theater
Unternehmen
1 token in lemmap
    Unternehmen -> Unternehmen
Regie
1 token in lemmap
    Regie -> Regie
SPD
1 token in lemmap
    SPD -> SPD
Rechtsanwalt
1 token in lemmap
    Rechtsanwalt -> Rechtsan

    Haus -> Haus
Artikel
1 token in lemmap
    Artikel -> Artikel
Buch
1 token in lemmap
    Buch -> Buch
Jugendliche
1 token in lemmap
    Jugendliche -> Jugendlichen
Abs.
1 token in lemmap
    Abs. -> Abs.
Patient
1 token in lemmap
    Patient -> Patienten
Vertrag
1 token in lemmap
    Vertrag -> Vertrag
Fisch
1 token in lemmap
    Fisch -> Fisch
Mann
1 token in lemmap
    Mann -> Mann
Film
1 token in lemmap
    Film -> Film
Saarland
1 token in lemmap
    Saarland -> Saarland
Nr.
1 token in lemmap
    Nr. -> Nr.
Wasser
1 token in lemmap
    Wasser -> Wasser
Sekt
1 token in lemmap
    Sekt -> Sekte
Lernhilfe
1 token in lemmap
    Lernhilfe -> Lernhilfe
EUR
1 token in lemmap
    EUR -> EUR
Prozent
1 token in lemmap
    Prozent -> Prozent
Kläger
1 token in lemmap
    Kläger -> Kläger
Ernährung
1 token in lemmap
    Ernährung -> Ernährung
Deutschland
1 token in lemmap
    Deutschland -> Deutschland
Polizei
1 token in lemmap
    Polizei -> Polizei
Wort
1 token in lemmap
    Wort -> Wort
S

    Frau -> Frau
Punkt
1 token in lemmap
    Punkt -> Punkt
Mutter
1 token in lemmap
    Mutter -> Mutter
Studentenjob
1 token in lemmap
    Studentenjob -> Studentenjobs
Katze
1 token in lemmap
    Katze -> Katze
Gott
1 token in lemmap
    Gott -> Gott
Insel
1 token in lemmap
    Insel -> Insel
Abs.
1 token in lemmap
    Abs. -> Abs.
Mendelssohn
1 token in lemmap
    Mendelssohn -> Mendelssohn
KeepSchool
1 token in lemmap
    KeepSchool -> KeepSchool
Opfer
1 token in lemmap
    Opfer -> Opfer
Thema
1 token in lemmap
    Thema -> Thema
Staat
1 token in lemmap
    Staat -> Staat
Farbe
1 token in lemmap
    Farbe -> Farbe
Israeli
1 token in lemmap
    Israeli -> Israelis
Beispiel
1 token in lemmap
    Beispiel -> Beispiel
Vorgesetzter
1 token in lemmap
    Vorgesetzter -> Vorgesetzten
Bremen
1 token in lemmap
    Bremen -> Bremen
Sinn
1 token in lemmap
    Sinn -> Sinne
Hausaufgabenhilfe
1 token in lemmap
    Hausaufgabenhilfe -> Hausaufgabenhilfe
Mutter
1 token in lemmap
    Mutter -> M

1 token in lemmap
    Prozent -> Prozent
Mannschaft
1 token in lemmap
    Mannschaft -> Mannschaft
Digest
1 token in lemmap
    Digest -> Digest
Recht
1 token in lemmap
    Recht -> Recht
Geschichte
1 token in lemmap
    Geschichte -> Geschichte
Bericht
1 token in lemmap
    Bericht -> Bericht
Rektor
1 token in lemmap
    Rektor -> Rektor
Ei
1 token in lemmap
    Ei -> Eier
AGPF
1 token in lemmap
    AGPF -> AGPF
Aufenthaltserlaubnis
1 token in lemmap
    Aufenthaltserlaubnis -> Aufenthaltserlaubnis
Betrag
1 token in lemmap
    Betrag -> Betrag
Anleger
1 token in lemmap
    Anleger -> Anleger
Anspruch
1 token in lemmap
    Anspruch -> Anspruch
Medikation
1 token in lemmap
    Medikation -> Medikation
Ziel
1 token in lemmap
    Ziel -> Ziel
Täter
1 token in lemmap
    Täter -> Täter
Stelle
1 token in lemmap
    Stelle -> Stelle
Sinn
1 token in lemmap
    Sinn -> Sinne
Volk
1 token in lemmap
    Volk -> Volk
Solinger
1 token in lemmap
    Solinger -> Solinger
Urt
1 token in lemmap
    Ur

    Zelle -> Zellen
Fassung
1 token in lemmap
    Fassung -> Fassung
AP
1 token in lemmap
    AP -> AP
Mobilfunk
1 token in lemmap
    Mobilfunk -> Mobilfunk
Kraft
1 token in lemmap
    Kraft -> Kraft
Kind
1 token in lemmap
    Kind -> Kinder
Recht
1 token in lemmap
    Recht -> Recht
Arbeitnehmer
1 token in lemmap
    Arbeitnehmer -> Arbeitnehmer
Aktiengesellschaft
1 token in lemmap
    Aktiengesellschaft -> Aktiengesellschaft
Republik
1 token in lemmap
    Republik -> Republik
Zeit
1 token in lemmap
    Zeit -> Zeit
Beschluss
1 token in lemmap
    Beschluss -> Beschlüsse
Gericht
1 token in lemmap
    Gericht -> Gericht
Mann
1 token in lemmap
    Mann -> Mann
Mumia
1 token in lemmap
    Mumia -> Mumia
AStA
1 token in lemmap
    AStA -> AStA
Stück
1 token in lemmap
    Stück -> Stück
Juni
1 token in lemmap
    Juni -> Juni
Bereich
1 token in lemmap
    Bereich -> Bereich
Folge
1 token in lemmap
    Folge -> Folge
Frage
1 token in lemmap
    Frage -> Frage
Eigentum
1 token in lemmap
   

    Regisseur -> Regisseur
Veröffentlichung
1 token in lemmap
    Veröffentlichung -> Veröffentlichung
Herausgeber
1 token in lemmap
    Herausgeber -> Herausgeber
Geschmack
1 token in lemmap
    Geschmack -> Geschmack
Bundesregierung
1 token in lemmap
    Bundesregierung -> Bundesregierung
Einreise
1 token in lemmap
    Einreise -> Einreise
Hamer
1 token in lemmap
    Hamer -> Hamer
Dollar
1 token in lemmap
    Dollar -> Dollar
Revision
1 token in lemmap
    Revision -> Revision
Stress
1 token in lemmap
    Stress -> Stress
Umwelt
1 token in lemmap
    Umwelt -> Umwelt
Haus
1 token in lemmap
    Haus -> Haus
Grafik
1 token in lemmap
    Grafik -> Grafik
Anspruch
1 token in lemmap
    Anspruch -> Anspruch
Kreuz
1 token in lemmap
    Kreuz -> Kreuz
Foto
1 token in lemmap
    Foto -> Fotos
Rückkehr
1 token in lemmap
    Rückkehr -> Rückkehr
Abb.
1 token in lemmap
    Abb. -> Abb.
Lehrplan
1 token in lemmap
    Lehrplan -> Lehrpläne
UB
1 token in lemmap
    UB -> UB
Stoff
1 token in lemma

1 token in lemmap
    Verwaltung -> Verwaltung
Mitte
1 token in lemmap
    Mitte -> Mitte
Meter
1 token in lemmap
    Meter -> Meter
Erzählung
1 token in lemmap
    Erzählung -> Erzählung
Kindertageseinrichtung
1 token in lemmap
    Kindertageseinrichtung -> Kindertageseinrichtungen
Land
1 token in lemmap
    Land -> Land
Folge
1 token in lemmap
    Folge -> Folge
Stadion
1 token in lemmap
    Stadion -> Stadion
Zeit
1 token in lemmap
    Zeit -> Zeit
Bischof
1 token in lemmap
    Bischof -> Bischof
Figur
1 token in lemmap
    Figur -> Figuren
Gruppe
1 token in lemmap
    Gruppe -> Gruppe
2_.
3 is phrase
2
.
2_.
    2_. -> 2.
Tee
1 token in lemmap
    Tee -> Tee
Scientology
1 token in lemmap
    Scientology -> Scientology
Deutschland
1 token in lemmap
    Deutschland -> Deutschland
Verbindlichkeit
1 token in lemmap
    Verbindlichkeit -> Verbindlichkeiten
Nummer
1 token in lemmap
    Nummer -> Nummer
Recht
1 token in lemmap
    Recht -> Recht
Mensch
1 token in lemmap
    Mensch -> Mens

term0           term1            ...                         term18                    term19
dataset param_id nb_topics topic_idx                                                 ...                                                         
dewac1  e42      100       0                       Partei             PDS            ...                       Homepage                    Stalin
                           1                        König            Frau            ...                     Prinzessin                     Nacht
                           2          Lebenspartnerschaft   Lebenspartner            ...                      Erklärung            Homosexualität
                           3                       BetrVG     Betriebsrat            ...                            BAG                       aaO
                           4                 Gesellschaft    Aufsichtsrat            ...                        Satzung                 Wahllokal
                           5                      Artikel          Gesetz            ...                   Abgeordneten                   Vertrag
                           6                       Kirche         Bischof            ...              Johannes Paul II.                    Dienst
                           7                 Entscheidung       Verfahren            ...                          Frist           Voraussetzungen
                           8                      Richter             Dr.            ...                       Tatsache                Verbrechen
                           9                  Zivildienst     Wehrpflicht            ...                    Grundgesetz                   Einsatz
                           10                    Menschen         München            ...                          Frage             Veranstaltung
                           11                      Physik             Bio            ...                           Land               Deutschland
                           12                       Musik            Band            ...                        Melodie                      Hits
                           13                     Theater           Stück            ...                         August              Käfighaltung
                           14                 Unternehmen          Kunden            ...                        Zukunft                  Anbieter
                           15                       Regie          Thomas            ...                          Platz                 Armstrong
                           16                         SPD          Grünen            ...                      Politiker                   CDU/CSU
                           17                Rechtsanwalt            Erbe            ...             Patientenverfügung             Maria Altmann
                           18                       Leute            Zeit            ...                         Moment                      Frau
                           19                  Richtlinie        Regelung            ...                    Gesetzgeber                 Maßnahmen
                           20                   Bewerbung           Frage            ...                    Anschreiben                      Lena
                           21                       Spiel         Spieler            ...                         Würfel                  Beispiel
                           22                        Frau           Vater            ...                           Ende                     Paris
                           23                         Dr.           Prof.            ...                           Zeit               Ausstellung
                           24                       Tiere            Hund            ...                          Teich                    Futter
                           25                    Menschen           Leben            ...                      

In [17]:
df_unlemm.to_csv(join(LDA_PATH, 'noun', 'bow', 'topics', 'dewac1_topics_unlemmatized.csv'))

In [1]:
for i in range(1, 100):
    print(f'<question id="DE{i:02d}" intID="{i*2}" />')

<question id="DE01" intID="2" />
<question id="DE02" intID="4" />
<question id="DE03" intID="6" />
<question id="DE04" intID="8" />
<question id="DE05" intID="10" />
<question id="DE06" intID="12" />
<question id="DE07" intID="14" />
<question id="DE08" intID="16" />
<question id="DE09" intID="18" />
<question id="DE10" intID="20" />
<question id="DE11" intID="22" />
<question id="DE12" intID="24" />
<question id="DE13" intID="26" />
<question id="DE14" intID="28" />
<question id="DE15" intID="30" />
<question id="DE16" intID="32" />
<question id="DE17" intID="34" />
<question id="DE18" intID="36" />
<question id="DE19" intID="38" />
<question id="DE20" intID="40" />
<question id="DE21" intID="42" />
<question id="DE22" intID="44" />
<question id="DE23" intID="46" />
<question id="DE24" intID="48" />
<question id="DE25" intID="50" />
<question id="DE26" intID="52" />
<question id="DE27" intID="54" />
<question id="DE28" intID="56" />
<question id="DE29" intID="58" />
<question id="DE30

In [55]:
with open('/home/andreas/Dropbox/Desktop/rerank_ratings/head.xml', 'r') as fp:
    head = fp.read()
with open('/home/andreas/Dropbox/Desktop/rerank_ratings/mid.xml', 'r') as fp:
    mid = fp.read()
with open('/home/andreas/Dropbox/Desktop/rerank_ratings/tail.xml', 'r') as fp:
    tail = fp.read()

tpx = topics_loader.topics
tpx

def topic_to_xml(topic):
    items = []
    head = (
        f'<question>'
        f'<description>Topic {topic.name[3]+1}</description>\n'
        f'<class>checkbox</class>\n'
        f'<title>Topic {topic.name[3]+1}</title>\n'
        f'<explanation />\n'
        f'<inputPosition>right</inputPosition>\n'
        f'<attributes.specific program="ofb">\n'
        f'<attr id="id">1</attr>\n'
        f'<attr id="pos">1</attr>\n'
        f'<attr id="metrics.gap">1</attr>\n'
        f'<attr id="restrictions.answered">yes</attr>\n'
        f'<attr id="composition">rows</attr>\n'
        f'<attr id="character">cards</attr>\n'
        f'<attr id="columns">1</attr>\n'
        f'<attr id="metrics.width_default">0</attr>\n'
        f'</attributes.specific>\n'
    )
    items.append(head)
    for i, term in enumerate(topic, 1):
        items.append(
            f'<item>\n'
            f'<text>{term}</text>\n'
            f'<attributes.specific program="ofb">\n'
            f'<attr id="id">{i}</attr>\n'
            f'<attr id="pos">{i}</attr>\n'
            f'</attributes.specific>\n'
            f'</item>\n'
        )
    tail = '</question>\n'
    items.append(tail)
    items = ''.join(items)
    return items

tpx = tpx.apply(topic_to_xml, axis=1)
questions = ''.join(tpx)

pages = []
for i in range(1, len(tpx)+1):
    page = (
        f'\n'
        f'<!-- Seite {i} -->\n'
        f'<page intID="{i*2-1}">\n'
        f'<question id="DE{i:02d}" intID="{i*2}" />\n'
        f'</page>\n'
    )
    pages.append(page)
pages = ''.join(pages)
    
questionnair = head + questions + mid + pages +tail

with open('/home/andreas/Dropbox/Desktop/rerank_ratings/questionnair.xml', 'w') as fp:
    fp.write(questionnair)
    
print(questionnair)

<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE surveyContent SYSTEM "https://www.soscisurvey.de/templates/doctype.survey.dtd">
<surveyContent version="2.4">
<program>sosci</program>
<version>3.1.06-i</version>
<timestamp>2018-11-30 00:33:14</timestamp>
<title>dewac ranking</title>
<description />
<language>deu</language>
<attributes.specific program="ofb">
<attr id="language">deu</attr>
</attributes.specific>
<section>
<title>dewac ranking</title>
<attributes.specific program="ofb">
<attr id="id">DE</attr>
</attributes.specific>
<question><description>Topic 1</description>
<class>checkbox</class>
<title>Topic 1</title>
<explanation />
<inputPosition>right</inputPosition>
<attributes.specific program="ofb">
<attr id="id">1</attr>
<attr id="pos">1</attr>
<attr id="metrics.gap">1</attr>
<attr id="restrictions.answered">yes</attr>
<attr id="composition">rows</attr>
<attr id="character">cards</attr>
<attr id="columns">1</attr>
<attr id="metrics.width_default">0</attr>
</attributes.specif

In [2]:
topics = load('dewac1', 'topics', 'e42', 'ref', 'c_npmi', 100)
topics_values = topics.values
wiki_dict = load('dict', 'dewiki', 'unfiltered')
wiki_texts = load('texts', 'dewiki')
#wiki_dict = Dictionary(wiki_texts)

#wiki_dict.save(join(LDA_PATH, 'noun', 'dewiki_noun_bow_unfiltered.dict'))

purpose topics
dataset dewac_nbfiles01
Reading ../data/preprocessed/LDAmodel/noun/Reranker/dewac_nbfiles01_topic-candidates.csv
purpose dict
dataset dewiki
loading dict from ../data/preprocessed/LDAmodel/noun/dewiki_noun_bow_unfiltered.dict
purpose texts
dataset dewiki
loading texts from ../data/preprocessed/LDAmodel/noun/dewiki_noun_bow_texts.json


In [4]:
in_dict = topics.applymap(lambda x: x in wiki_dict.token2id)
isstr = lambda x: isinstance(x, str)

oov = topics[~in_dict]
oov = oov.apply(set)
oov = set().union(*oov)
oov = sorted(map(lambda x: [x], filter(isstr, oov)))

if oov:
    wiki_texts += oov
    wiki_dict.add_documents(oov)
    wiki_dict.id2token[1]

oov

[]

In [5]:
metrics = ['u_mass', 'c_v', 'c_npmi', 'c_uci']

scores = dict()
for metric in metrics:
    print(metric)
    cm = CoherenceModel(
        topics=topics_values,
        dictionary=wiki_dict,
        texts=wiki_texts[-1000:],
        coherence=metric,
        topn=10,
        window_size=20,
        processes=4
    )
    coherence_scores = cm.get_coherence_per_topic(with_std=True, with_support=True)
    scores[metric] = coherence_scores
scores

u_mass
c_v


/home/andreas/bin/anaconda3/lib/python3.6/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:196: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.logg(numerator / denominator)
/home/andreas/bin/anaconda3/lib/python3.6/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:317: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


c_npmi
c_uci


{'u_mass': [-16.52285158236896,
  -5.018724105025476,
  -17.441237995523498,
  -9.268998779271525,
  -12.770278227951561,
  -2.6366163489737953,
  -7.692643639563276,
  -13.515293901885618,
  -17.707929546921427,
  -14.575910508831468,
  -14.304108277910478,
  -10.370501492902703,
  -3.3497821844465907,
  -15.108326797913673,
  -5.576959372378067,
  -14.065948492317705,
  -11.975560840245343,
  -15.625352774898998,
  -5.573071816664561,
  -15.21776189155979,
  -17.595373103993982,
  -11.0085223412414,
  -1.617084652262527,
  -8.627527044065879,
  -17.11259042455439,
  -5.508700788621,
  -4.6770955719731315,
  -10.714732675587834,
  -15.957461349425607,
  -12.15995101975449,
  -8.778424065511631,
  -17.49447532544521,
  -4.622747374144896,
  -9.32554600453244,
  -11.652365178481308,
  -4.091617542054341,
  -12.284028857614668,
  -17.288200308415757,
  -6.4779078237663,
  -14.266146323719148,
  -5.34301206442187,
  -13.19749749136664,
  -9.626626064652505,
  -16.967016315439366,
  -16.94

In [ ]:
#wiki_dict.id2token['Partei']
#wiki_dict.token2id['Partei']
#topics[[]].values.tolist()
#wiki_texts[-100:]

In [1]:
df = pd.DataFrame(scores)
df.index = topics.index

NameError: name 'pd' is not defined

In [ ]:
topics['c_uci_wiki'] = coherence_scores
topics

In [ ]:
df[df.description.str.lower() == 'redirect']

In [ ]:
rerank_path = join(LDA_PATH, 'topics')
PARAMS = 'a42'
TOPN = 20
NBTOPIC = 10
PATH = 'LDAmodel'
DATASET = DATASETS['E']

def load_model(dataset=DATASET, param_id=PARAMS, nb_topics=NBTOPIC, subdir=PATH):
    """
    Load an LDA model.
    """
    model_filename = f'{dataset}_LDAmodel_{param_id}_{nb_topics}'
    in_path = join(ETL_PATH, subdir, param_id, model_filename)
    print('Loading model from', in_path)
    ldamodel = LdaModel.load(in_path)
    return ldamodel

def df_topics(ldamodel, dataset=DATASET, topn=TOPN, nb_topics=NBTOPIC):
    dictionary = ldamodel.id2word
    topics = [
        [dataset] +
        [dictionary[term[0]] for term in ldamodel.get_topic_terms(i, topn=topn)]
        for i in range(nb_topics)
    ]
    df = pd.DataFrame(topics, columns=['dataset'] + ['term' + str(i) for i in range(topn)])
    return df

def save_topics(df, dataset=DATASET, param_id=PARAMS, nb_topics=NBTOPIC, subdir=PATH):
    out_path = join(out_dir, '{}_{}_{}_{}'.format(dataset, model_name, param_id, nb_topics))
    logger.info('saving to ' + out_path)
    df.to_csv(out + '.csv')
